# Banka z SQLAlchemy

Knjižnico `sqlalchemy` namestimo z ukazom
```bash
python -m pip install sqlalchemy
```
oziroma (če nimamo administratorskih pravic)
```bash
python -m pip install --user sqlalchemy
```

In [ ]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy import create_engine

Definirajmo naše razrede (entitetne tipe).

In [ ]:
Base = declarative_base()

class Kraj(Base):
    # Tako se bo imenavala tabela v bazi
    __tablename__ = 'kraj'
    
    # Atributi (lastnosti), ki sledijo se preslikajo v stolpce
    posta = Column(Integer, primary_key=True)
    kraj = Column(String(250))

class Oseba(Base):
    __tablename__ = 'oseba'
    
    id = Column(Integer, primary_key=True)               # Primarni ključ
    ime = Column(String(250), nullable=False)            # Ne sme biti NULL
    kraj_id = Column(Integer, ForeignKey('kraj.posta')) # tuji ključ
    # Lastnost, ki odraža povezavo do tabele kraj.
    # Ker je enolično določena, ne rabimo dodatnih parametrov.
    kraj = relationship(Kraj)

Generirajmo podatkovno bazo v SQLite (datoteko `banka-orm.db`) oziroma se z njo povežimo, če datoteka že obstaja.

Če želimo izpise stavkov SQL, nastavimo parameter `echo=True`. Tega v produkciji **ne** nastavimo.

In [ ]:
engine = create_engine('sqlite:///banka-orm.db', echo=True)

Povežimo `engine` na razred `Base`.

In [ ]:
Base.metadata.bind = engine

Tabele na bazi v tem trenutku še niso ustvarjene. Ustvarimo jih z naslednjim klicem.

In [ ]:
Base.metadata.create_all(engine)

Odprimo generator sej in ustvarimo novo sejo.

In [ ]:
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

Ustvarimo nov objekt razreda `Kraj` in ga dodamo v sejo.

In [ ]:
nov_kraj = Kraj(posta=1000, kraj='Ljubljana')
session.add(nov_kraj)

Metoda `commit` shrani (spremenjene) objekte v seji v podatkovno bazo.

In [ ]:
session.commit()

Ustvarimo nov objekt razreda `Oseba` in ga preko lastnosti `kraj` povežemo s prej ustvarjenim krajem.

In [ ]:
nova_oseba = Oseba(ime='Janez', kraj=nov_kraj)
session.add(nova_oseba)
session.commit()

Izvedimo poizvedbo `SELECT * FROM oseba`.

In [ ]:
session.query(Oseba).all()

Vrnimo prvi kraj in ga izpišimo.

In [ ]:
kraj1 = session.query(Kraj).first()
print(kraj1)
print("KRAJ:", kraj1.kraj)

Poiščimo vse osebe v kraju `kraj1` in jih izpišimo kot objekte.

In [ ]:
session.query(Oseba).filter(Oseba.kraj == kraj1).all()

Poiščimo eno tako osebo ter jo vrnimo in izpišimo.

In [ ]:
oseba1 = session.query(Oseba).filter(Oseba.kraj == kraj1).one()
print(oseba1)
print("IME:", oseba1.ime)
print("KRAJ:", oseba1.kraj)